In [24]:
# Import packages
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader as pdr
import statsmodels.api as sm
import math as ma

In [25]:
# Load Fama-French factor data
ff3f = pdr.DataReader('F-F_Research_Data_Factors', 'famafrench', '1950-01-01', '2020-12-31')[0]/100
ff3f.head(5)

,Mkt-RF,SMB,HML,RF
Date,,,,
1950-01,0.0170,0.0333,0.0012,0.0009
1950-02,0.0148,0.0003,-0.0079,0.0009
1950-03,0.0126,-0.0144,-0.0281,0.0010
1950-04,0.0394,0.0194,0.0130,0.0009
1950-05,0.0431,-0.0210,0.0046,0.0010


In [26]:
    # Download monthly prices (keep only Adjusted Close prices)
ticker = 'MSFT'    
    
firm_prices = yf.download(ticker, '2016-01-01', '2020-12-31', interval = '1mo')['Adj Close'].dropna().to_frame()

    # Calculate monthly returns, drop missing, convert from Series to DataFrame
firm_ret = firm_prices.pct_change().dropna()

    # Rename "Adj Close" to "TSLA"
firm_ret.rename(columns = {'Adj Close': 'Returns'}, inplace = True)

    # Convert index to monthly period date
firm_ret.index = firm_ret.index.to_period('M')
firm_ret.head(2)

[*********************100%%**********************]  1 of 1 completed


,Returns
Date,
2016-02,-0.076420
2016-03,0.093289


In [27]:
# Merge the two datasets
data = firm_ret.join(ff3f)
data['const'] = 1
data.head(2)

,Returns,Mkt-RF,SMB,HML,RF,const
Date,,,,,,
2016-02,-0.076420,-0.0007,0.0073,-0.0057,0.0002,1
2016-03,0.093289,0.0696,0.0082,0.0119,0.0002,1


In [28]:
# Set up the data
    # Dependent variable (left side of the equal sign)
y = data['Returns'] - data['RF']
y.head(2)

Date
2016-02   -0.076620
2016-03    0.093089
Freq: M, dtype: float64

In [29]:
    # Independent variable(s) (right side of the equal sign)
X = data[['const','Mkt-RF']]
X.head(2)

,const,Mkt-RF
Date,,
2016-02,1,-0.0007
2016-03,1,0.0696


In [30]:
# Run single factor regression and store results in "res" object
res = sm.OLS(y,X).fit()
print(res.summary())
print("Reminder that 1 - Rsquared equals the percentage of the firms total risk that can be diversified away (idiosyncratic)")

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.445
Model:                            OLS   Adj. R-squared:                  0.435
Method:                 Least Squares   F-statistic:                     45.67
Date:                Wed, 13 Mar 2024   Prob (F-statistic):           8.05e-09
Time:                        20:46:40   Log-Likelihood:                 107.47
No. Observations:                  59   AIC:                            -210.9
Df Residuals:                      57   BIC:                            -206.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0153      0.005      2.813      0.0

In [31]:
#Adjusted Betas
l = 0.33
hBeta = res.params[1]
Adj_Beta = hBeta * (1-l) + l
print(f"The Adjusted Beta For {ticker} is: {Adj_Beta}")

The Adjusted Beta For MSFT is: 0.840518490961532


In [32]:
#Single Factor Cost of Equity
Emrp = sum(ff3f['Mkt-RF']) / len(ff3f['Mkt-RF'])
Rf = 0.0001
COE = (Rf + hBeta * Emrp) *12 * 100
COE_r = round(COE, 3)
print(f"The Annualized Cost of Equity for {ticker} is: {COE_r}%")

The Annualized Cost of Equity for MSFT is: 6.228%


In [33]:
#Estimate the Fama-French three factor model using the data gathered above


# Set up X variables
X3 = data[['const','Mkt-RF','SMB','HML']]
# Run regression
res3 = sm.OLS(y,X3).fit()
print(res3.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.554
Model:                            OLS   Adj. R-squared:                  0.529
Method:                 Least Squares   F-statistic:                     22.74
Date:                Wed, 13 Mar 2024   Prob (F-statistic):           1.05e-09
Time:                        20:46:40   Log-Likelihood:                 113.91
No. Observations:                  59   AIC:                            -219.8
Df Residuals:                      55   BIC:                            -211.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0107      0.005      2.063      0.0

In [37]:
# Total risk of our stock (variance) in 3 factor (also works in single factor)
tot_risk  = y.var()
print(f"The total risk of {ticker} equals: {tot_risk}")

# Systematic Risk
beta = res.params['Mkt-RF']
sys_risk  = (beta**2) * data['Mkt-RF'].var() 
print(f"The systematic risk equals: {sys_risk}")

#Idiosyncratic Risk 
idio_risk = tot_risk - sys_risk
print(f"The idiosyncratic risk equals: {idio_risk}")

The total risk of MSFT equals: 0.0028077908811993627
The systematic risk equals: 0.0012489551941581379
The idiosyncratic risk equals: 0.0015588356870412249


In [35]:
# Print as percentages of total risk
pct_sys_risk = sys_risk / tot_risk
pct_idio_risk = idio_risk / tot_risk
print(f'\n percent systematic risk = {pct_sys_risk: .4f} \n percent idiosyncratic risk = {pct_idio_risk: .4f}')


 percent systematic risk =  0.4448 
 percent idiosyncratic risk =  0.5552


In [36]:
#Cost of Equity in 3 factor
#We already have our risk free rate variabe "Rf" 
#and the expected Market RIsk Premium "Ermp" and beta for the stock "hBeta"
#We now need the exp ret for our other two factors and their betas
#also we need to update the beta "hBeta" to "h3Beta"

#Expected Returns
ErSMB = sum(ff3f['SMB']) / len(ff3f['SMB'])
ErHML = sum(ff3f['HML']) / len(ff3f['HML'])

#SMB, HML, and 3h Betas
Bhml = res3.params[3]
Bsmb = res3.params[2]
h3Beta = res3.params[1]
print(f" HML - {Bhml} and SMB - {Bsmb} and the market beta {h3Beta}")


f3COE = Rf + (h3Beta * Emrp) + (Bsmb * ErSMB) + (Bhml * ErHML)
Af3COE = (f3COE * 12) * 100


print(f"The **Annualized*** Cost of Equity for {ticker} is: {Af3COE.round(3)}%")


 HML - -0.36598822658177455 and SMB - -0.4716492557868956 and the market beta 0.9425620895335998
The **Annualized*** Cost of Equity for MSFT is: 5.668%
